In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from keras.models import Model, load_model
from keras.layers import Input, Average
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator

# -------------------------------
# Set Image Properties
# -------------------------------
img_height, img_width = 224, 224
batch_size = 16
epochs = 30


# -------------------------------
# Load Pretrained Models (Segmented)
# -------------------------------
model_1 = load_model('/content/drive/MyDrive/model/mobileNetV2_ensemble_model.h5')
model_1._name = 'mobile_net'

model_2 = load_model('/content/drive/MyDrive/train_test_val/Dense121_64_model.h5')
model_2._name = 'dense_net'

model_3 = load_model('/content/drive/MyDrive/Ensemble_models/VGG16-18-0.9000.hdf5')
model_3._name = 'vgg16_net'

# -------------------------------
# Create Ensemble Model
# -------------------------------
models = [model_1, model_2, model_3]
model_input = Input(shape=(img_height, img_width, 3))
model_outputs = [model(model_input) for model in models]
ensemble_output = Average()(model_outputs)

ensemble_model = Model(inputs=model_input, outputs=ensemble_output, name='ensemble_model_segmented')
ensemble_model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# -------------------------------
# Train Ensemble Model
# -------------------------------
history = ensemble_model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=epochs
)

# -------------------------------
# Evaluate on Test Set
# -------------------------------
test_loss, test_accuracy = ensemble_model.evaluate(test_generator)
print(f"\nTest Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")

# -------------------------------
# Plot Accuracy & Loss
# -------------------------------
def plot_metrics(history):
    plt.figure(figsize=(12, 4))

    # Accuracy
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], 'r', label='Train Accuracy')
    plt.plot(history.history['val_accuracy'], 'b', label='Validation Accuracy')
    plt.title('Model Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.ylim(0.8, 1.0)

    # Loss
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], 'r', label='Train Loss')
    plt.plot(history.history['val_loss'], 'b', label='Validation Loss')
    plt.title('Model Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.ylim(0, 0.5)

    plt.tight_layout()
    plt.show()

plot_metrics(history)

# -------------------------------
# Save Ensemble Model
# -------------------------------
model_save_path = '/content/drive/MyDrive/Ensemble_models/trio_ensemble.h5'
ensemble_model.save(model_save_path)
print("✅ Ensemble model saved successfully at:", model_save_path)